In [7]:
import urllib.request
from bs4 import BeautifulSoup
from flask import Flask, render_template, request
import sqlite3

app = Flask(__name__)

conn = sqlite3.connect("music.db")
cursor = conn.cursor()


In [73]:
#### tablename에 table이 있는 경우 그냥 데이터 삭제후 다시 웹 크롤링
def chart_balad_save():
    url = "https://www.melon.com/chart/day/index.htm?classCd=GN0200" # 멜론 발라드 차트 url 
    hdr = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.61 Safari/537.36"}
    req = urllib.request.Request(url, headers=hdr)
    html = urllib.request.urlopen(req).read()
    soup = BeautifulSoup(html, 'html.parser')

    conn = sqlite3.connect("music.db")
    cursor = conn.cursor()


    
    lst50 = soup.select('.lst50, .lst100')

    genre = []

    paging = soup.find('div', class_ = 'calendar_prid').get_text().replace("\n","").replace(".","_")
    genre.append([paging])

    cursor.execute("DELETE FROM {}".format(genre[0]))

    melonList = []
    for i in lst50:
        ranking = (i.select_one("td > div > span.rank").text+"위") # 멜론 사이트에서 발라드 순위 가져와서 temp에 추가
        title = (i.select_one("td > div > div > div.ellipsis.rank01").a.text) # 멜론 사이트에서 노래 제목 가져오서 temp에 추가
        singer = (i.select_one('td > div > div > div.ellipsis.rank02').a.text) # 멜론 사이트에서 가수 이름 가져와서 temp에 추가
        album = (i.select_one('td > div > div > div.ellipsis.rank03').a.text) # 멜론 사이트에서 앨범 이름 가져와서 temp에 주가
        url_title = "https://www.youtube.com/results?search_query={}".format((i.select_one('td > div > div > div.ellipsis.rank03').a.text))
        url_singer = "https://www.youtube.com/results?search_query={}".format((i.select_one('td > div > div > div.ellipsis.rank02').a.text))
        melonList.append([ranking, title, singer, album]) # 멜론리스트에 temp에 있는 것들 추가
        
        inser='insert into {} (ranking, title, singer, album, youtube_title, youtube_singer) values(?,?,?,?,?,?)'.format(genre[0])
        cursor.execute(inser,(ranking, title, singer, album, url_title, url_singer))
    conn.commit()


#### teblename에 없는 table일 경우 table을 생성하고 데이터 저장
def chart_balad_save1():
    url = "https://www.melon.com/chart/day/index.htm?classCd=GN0200" # 멜론 발라드 차트 url 
    hdr = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.61 Safari/537.36"}
    req = urllib.request.Request(url, headers=hdr)
    html = urllib.request.urlopen(req).read()
    soup = BeautifulSoup(html, 'html.parser')

    conn = sqlite3.connect("music.db")
    cursor = conn.cursor()

    lst50 = soup.select('.lst50, .lst100')

    genre = []

    paging = soup.find('div', class_ = 'calendar_prid').get_text().replace("\n","").replace(".","_")
    genre.append([paging])

    cursor.execute('''insert into tablename (name) values(?)''', (paging))

    # inser='insert into tablename (name) values(?)'
    # cursor.execute(inser,(paging))
    conn.commit()


    insert = 'create table {} (ranking REAL, title TEXT, singer TEXT, album TEXT, youtube_title TEXT, youtube_singer TEXT)'.format(paging)
    cursor.execute(insert)
    conn.commit()   

    melonList = []
    for i in lst50:
        ranking = (i.select_one("td > div > span.rank").text+"위") # 멜론 사이트에서 발라드 순위 가져와서 temp에 추가
        title = (i.select_one("td > div > div > div.ellipsis.rank01").a.text) # 멜론 사이트에서 노래 제목 가져오서 temp에 추가
        singer = (i.select_one('td > div > div > div.ellipsis.rank02').a.text) # 멜론 사이트에서 가수 이름 가져와서 temp에 추가
        album = (i.select_one('td > div > div > div.ellipsis.rank03').a.text) # 멜론 사이트에서 앨범 이름 가져와서 temp에 주가
        url_title = "https://www.youtube.com/results?search_query={}".format((i.select_one('td > div > div > div.ellipsis.rank03').a.text))
        url_singer = "https://www.youtube.com/results?search_query={}".format((i.select_one('td > div > div > div.ellipsis.rank02').a.text))
        melonList.append([ranking, title, singer, album]) # 멜론리스트에 temp에 있는 것들 추가
        
        inser='insert into {} (ranking, title, singer, album, youtube_title, youtube_singer) values(?,?,?,?,?,?)'.format(paging)
        cursor.execute(inser,(ranking, title, singer, album, url_title, url_singer))
    conn.commit()


In [74]:
url = "https://www.melon.com/chart/day/index.htm?classCd=GN0200" # 멜론 발라드 차트 url 
hdr = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.61 Safari/537.36"}
req = urllib.request.Request(url, headers=hdr)
html = urllib.request.urlopen(req).read()
soup = BeautifulSoup(html, 'html.parser')


genre = []

paging = soup.find('div', class_ = 'calendar_prid').get_text().replace("\n","").replace(".","_")
genre.append([paging])


conn=sqlite3.connect('music.db')
conn.row_factory=sqlite3.Row
cursor=conn.cursor()

cursor.execute('select*from tablename where name =? ',(paging,))
row = cursor.fetchone()
conn.close()

if row!=None:
    if paging == row['name']:
        chart_balad_save()
else:
    chart_balad_save1()


ProgrammingError: Incorrect number of bindings supplied. The current statement uses 1, and there are 12 supplied.

In [66]:
# 멜론 발라드 차트 데이터를 가져와 데이터 베이스에 저장

url = "https://www.melon.com/chart/day/index.htm?classCd=GN0800" # 멜론 발라드 차트 url 
hdr = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.61 Safari/537.36"}
req = urllib.request.Request(url, headers=hdr)
html = urllib.request.urlopen(req).read()
soup = BeautifulSoup(html, 'html.parser')


lst50 = soup.select('.lst50, .lst100')

genre = []

paging = soup.find('div', class_ = 'calendar_prid').get_text().replace("\n","").replace(".","-")
genre.append([paging])

new = len(paging)

print(paging)
print(genre[0][0])
print(genre)
print(new)
def save():
    inser='insert into tablename (name) values(?)'
    cursor.execute(inser,(genre[0]))
    conn.commit()

def search():
    conn=sqlite3.connect('music.db')
    conn.row_factory=sqlite3.Row
    cursor=conn.cursor()

    cursor.execute('select*from tablename where name =? ',(paging,))
    row = cursor.fetchone()
    conn.close()

    if row!=None:
        if paging == row['name']:
            print("!")
    else:
        print("0")


2022-11-25포크/블루스
2022-11-25포크/블루스
[['2022-11-25포크/블루스']]
16


In [209]:
conn.close()